In [1]:
import os
os.environ["THEANO_FLAGS"] = 'cuda.root=/usr/local/cuda,floatX=float32,device=gpu0,force_device=False,lib.cnmem=.75'

import theano
print(theano.config.device)

import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time, logging
import mhcflurry.dataset
import fancyimpute

import sklearn.metrics
import sklearn.cross_validation
from collections import defaultdict
import numpy as np

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
ERROR:theano.sandbox.cuda:nvcc compiler not found on $PATH. Check your nvcc installation and try again.


Couldn't import dot_parser, loading of dot files will not be possible.
gpu0

Using Theano backend.
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
min_peptides_to_consider_allele = 10
max_ic50 = 50000
data_dir = "../data/"

In [3]:
all_train_data = mhcflurry.dataset.Dataset.from_csv(data_dir + "bdata.2009.mhci.public.1.txt")

In [4]:
all_train_data._df

species      allele  peptide_length   cv  \
allele     peptide                                                     
ELA-A1     GSQKLTTGNCNW        None      ELA-A1              12  TBD   
           HVKDETNTTEYW        None      ELA-A1              12  TBD   
           LVEDVTNTAEYW        None      ELA-A1              12  TBD   
           RVEDKTNTAEYW        None      ELA-A1              12  TBD   
           RVEDVKNTAEYW        None      ELA-A1              12  TBD   
           RVEDVTLTAEYW        None      ELA-A1              12  TBD   
           RVEDVTNKAEYW        None      ELA-A1              12  TBD   
           RVEDVTNTAELW        None      ELA-A1              12  TBD   
           RVEDVTNTAEYL        None      ELA-A1              12  TBD   
           RVEDVTNTAEYW        None      ELA-A1              12  TBD   
           RVEDVTNTALYW        None      ELA-A1              12  TBD   
           RVEDVTNTKEYW        None      ELA-A1              12  TBD   
           RVEKVTNTAEYW        None      ELA-A1              12  TBD   
           RVLDVTNTAEYW        None      ELA-A1              12  TBD   
Gogo-B0101 RRFVNVVPTF       gorilla  Gogo-B0101              10  TBD   
           ERYLKDQQL        gorilla  Gogo-B0101               9  TBD   
           GRFKLIVLY        gorilla  Gogo-B0101               9  TBD   
           IDFPKTFGW        gorilla  Gogo-B0101               9  TBD   
           IFFPKTFGW        gorilla  Gogo-B0101               9  TBD   
           IKFPKTFGW        gorilla  Gogo-B0101               9  TBD   
           ILFPKTFGW        gorilla  Gogo-B0101               9  TBD   
           INFPKTFGW        gorilla  Gogo-B0101               9  TBD   
           IRFPKTFGW        gorilla  Gogo-B0101               9  TBD   
           IRYPKTFGW        gorilla  Gogo-B0101               9  TBD   
           KRGILTLKY        gorilla  Gogo-B0101               9  TBD   
           KRKKAYADF        gorilla  Gogo-B0101               9  TBD   
           KRYKSIVKY        gorilla  Gogo-B0101               9  TBD   
           RRYQKSTEL        gorilla  Gogo-B0101               9  TBD   
           SRDKTIIMW        gorilla  Gogo-B0101               9  TBD   
H-2-DB     AAAAAAYAAM         mouse      H-2-DB              10  TBD   
...                             ...         ...             ...  ...   
Patr-B2401 SLYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SNYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SPYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SQYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SRYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SSYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           STYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SVYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           SYYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           TDATSILGI     chimpanzee  Patr-B2401               9  TBD   
           TDNSSPPAV     chimpanzee  Patr-B2401               9  TBD   
           TDYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           TEAMTRYSA     chimpanzee  Patr-B2401               9  TBD   
           TESTLSTAL     chimpanzee  Patr-B2401               9  TBD   
           TILGIGTVL     chimpanzee  Patr-B2401               9  TBD   
           VDFIPVENL     chimpanzee  Patr-B2401               9  TBD   
           VDILAGYGA     chimpanzee  Patr-B2401               9  TBD   
           VDKNPHNTA     chimpanzee  Patr-B2401               9  TBD   
           VDPNIRTGV     chimpanzee  Patr-B2401               9  TBD   
           VDVQYLYGV     chimpanzee  Patr-B2401               9  TBD   
           VDYLELDTI     chimpanzee  Patr-B2401               9  TBD   
           VEAQLHVWV     chimpanzee  Patr-B2401               9  TBD   
           VESENKVVI     chimpanzee  Patr-B2401               9  TBD

In [6]:
alleles = [
    "HLA-A0201",
    # "HLA-A0301",
    "HLA-A0203",
    "HLA-A2602",
    "HLA-A2603",
    # 'HLA-B7301',
]
#alleles = alleles[:1] + alleles[-1:]
#alleles = [allele for allele in all_train_data if len(all_train_data[allele].Y) >= min_peptides_to_consider_allele]

In [7]:
train_data = mhcflurry.dataset.Dataset(all_train_data._df.ix[all_train_data._df.allele.isin(alleles)])
train_data._df

species     allele  peptide_length   cv     peptide  \
allele    peptide                                                          
HLA-A0201 AAAKTPVIVV   human  HLA-A0201              10  TBD  AAAKTPVIVV   
          AAASSTHRKV   human  HLA-A0201              10  TBD  AAASSTHRKV   
          AACIVGCENV   human  HLA-A0201              10  TBD  AACIVGCENV   
          AADLTQIFEV   human  HLA-A0201              10  TBD  AADLTQIFEV   
          AAERGPGQML   human  HLA-A0201              10  TBD  AAERGPGQML   
          AAGIGILTVI   human  HLA-A0201              10  TBD  AAGIGILTVI   
          AAGLQDCTML   human  HLA-A0201              10  TBD  AAGLQDCTML   
          AAITDAAVAV   human  HLA-A0201              10  TBD  AAITDAAVAV   
          AAITLVVISV   human  HLA-A0201              10  TBD  AAITLVVISV   
          AANPHATFGV   human  HLA-A0201              10  TBD  AANPHATFGV   
          AASPMLYQLL   human  HLA-A0201              10  TBD  AASPMLYQLL   
          AASSTHRKVA   human  HLA-A0201              10  TBD  AASSTHRKVA   
          AASTLLYATV   human  HLA-A0201              10  TBD  AASTLLYATV   
          AATKRYPGVM   human  HLA-A0201              10  TBD  AATKRYPGVM   
          AAVEQILTSV   human  HLA-A0201              10  TBD  AAVEQILTSV   
          AAVIIMAINV   human  HLA-A0201              10  TBD  AAVIIMAINV   
          AAVTLNRIKI   human  HLA-A0201              10  TBD  AAVTLNRIKI   
          ADLMGYIPLV   human  HLA-A0201              10  TBD  ADLMGYIPLV   
          AEMKTDAATL   human  HLA-A0201              10  TBD  AEMKTDAATL   
          AERGPGQMLG   human  HLA-A0201              10  TBD  AERGPGQMLG   
          AFEFINSLLK   human  HLA-A0201              10  TBD  AFEFINSLLK   
          AIAACAMLLV   human  HLA-A0201              10  TBD  AIAACAMLLV   
          AIAWGAWGEV   human  HLA-A0201              10  TBD  AIAWGAWGEV   
          AIDFGNGADL   human  HLA-A0201              10  TBD  AIDFGNGADL   
          AIDRQVSVKL   human  HLA-A0201              10  TBD  AIDRQVSVKL   
          AIFNNRNLAA   human  HLA-A0201              10  TBD  AIFNNRNLAA   
          AIFTDASTVA   human  HLA-A0201              10  TBD  AIFTDASTVA   
          AIIAIVFVFI   human  HLA-A0201              10  TBD  AIIAIVFVFI   
          AIIRQRNDEI   human  HLA-A0201              10  TBD  AIIRQRNDEI   
          AIKQYGDIDL   human  HLA-A0201              10  TBD  AIKQYGDIDL   
...                      ...        ...             ...  ...         ...   
HLA-A2603 VERLKHGTF    human  HLA-A2603               9  TBD   VERLKHGTF   
          VMAASGAPF    human  HLA-A2603               9  TBD   VMAASGAPF   
          VTFFCVMTY    human  HLA-A2603               9  TBD   VTFFCVMTY   
          VYFVLTDRF    human  HLA-A2603               9  TBD   VYFVLTDRF   
          WESGAVLCV    human  HLA-A2603               9  TBD   WESGAVLCV   
          WFGHLASDW    human  HLA-A2603               9  TBD   WFGHLASDW   
          WHQARFEEL    human  HLA-A2603               9  TBD   WHQARFEEL   
          WLSVIAFGK    human  HLA-A2603               9  TBD   WLSVIAFGK   
          WRQEIGHPK    human  HLA-A2603               9  TBD   WRQEIGHPK   
          WRWKSQVTI    human  HLA-A2603               9  TBD   WRWKSQVTI   
          WTALMFAAY    human  HLA-A2603               9  TBD   WTALMFAAY   
          WTIGNRAPY    human  HLA-A2603               9  TBD   WTIGNRAPY   
          YELDLWGKI    human  HLA-A2603               9  TBD   YELDLWGKI   
          YERGNIIIF    human  HLA-A2603               9  TBD   YERGNIIIF   
          YGIGVGSGY    human  HLA-A2603               9  TBD   YGIGVGSGY   
          YHLGGIEGL    human  HLA-A2603               9  TBD   YHLGGIEGL   
          YHSQGSWYK    human  HLA-A2603               9  TBD   YHSQGSWYK   
          YLHDPLTPY    human  HLA-A2603               9  TBD   YLHDPLTPY   
          YMYAVSGAL    human  HLA-A2603               9  TBD   YMYAVSGAL   
          YQNEVTPEY    human  HLA-A2603               9  TBD   YQNEVT

In [9]:
#train_data = dict((allele, data)
#                  for (allele, data) in all_train_data.items()
#                  if len(data.Y) >= min_peptides_to_consider_allele)
train_data = dict((allele, all_train_data.groupby_allele_dictionary()[allele]) for allele in alleles)
print("Training data: %d / %d alleles" % (len(train_data), len(all_train_data)))

#test_data = mhcflurry.data.load_allele_datasets("/Users/tim/sinai/git/mhcflurry/bdata.2013.mhci.public.blind.1.txt")


Training data: 4 / 137654 alleles


In [10]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(y, y_pred, weights=None, sample_weight=None, threshold_nm=500):
    ic50_y = log_to_ic50(y)
    ic50_y_pred = log_to_ic50(y_pred) 
    return dict(
        auc=sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight),
        f1=sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight),
        tau=scipy.stats.kendalltau(y_pred, y)[0],
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def Xallele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def Xdf_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)

def Xget_shuffled_fields(allele_data):
    """
    Parameters
    ----------
    allele_data : mhcflurry.data.AlleleData
    
    Returns shuffled array fields
    """
    original_peptides = data.original_peptides
    X = data.X_index 
    Y = data.Y
    weights = data.weights
    
    # shuffle all the  samples!
    shuffle_indices = np.arange(len(X))
    np.random.shuffle(shuffle_indices)
    
    X = X[shuffle_indices]
    Y = Y[shuffle_indices]
    weights = weights[shuffle_indices]
    original_peptides = original_peptides[shuffle_indices]
    
    return X, Y, weights, original_peptides

def collapse_9mer_affinities(Y_9mer_true, Y_9mer_pred, original_peptides):
    """
    Parameters
    ----------
    Y_9mer_true : np.array of float
        True regression target values for 9mers extracted from longer/shorter peptides
    
    Y_9mer_pred : np.array of float
        Predicted values for 9mers
    
    original_peptides : np.array of str
        Original peptides of varying length that 9mers were extracted from.
    """
    # collapse multiple 9mer predictions and measured values into 
    # smaller set of predictions for peptides of varying lengths
    peptide_to_true_affinity_dict = defaultdict(list)
    peptide_to_predicted_affinity_dict = defaultdict(list)
    for i, p in enumerate(original_peptides):
        peptide_to_true_affinity_dict[p].append(Y_9mer_true[i])
        peptide_to_predicted_affinity_dict[p].append(Y_9mer_pred[i])

    unique_peptides = list(sorted(set(peptide_to_predicted_affinity_dict.keys())))
    print("-- # unique peptides = %d" % (len(unique_peptides),))
    Y_true = np.array([
            np.mean(peptide_to_true_affinity_dict[p]) for p in unique_peptides ])
    Y_pred = np.array([
            np.mean(peptide_to_predicted_affinity_dict[p]) for p in unique_peptides])
    return Y_true, Y_pred

In [11]:
dropout_probabilities = [0.0, 0.5]

embedding_output_dims = [32, 8]
#embedding_output_dims = [4, 32]

#layer_sizes = [[4], [8], [16], [64], [128]]
layer_sizes_list = [[128], [4]]

activations = ["tanh"]

models_params_list = []
#for pretrain_decay in ["1 / (1+epoch)**2", "numpy.exp(-epoch)"]:
for fraction_negative in [0, .05, .10, .25]:
    for dropout_probability in dropout_probabilities:
        for embedding_output_dim in embedding_output_dims:
            for layer_sizes in layer_sizes_list:
                for activation in activations:
                    models_params_list.append(dict(
                        # pretrain_decay=pretrain_decay,
                        fraction_negative=fraction_negative,
                        dropout_probability=dropout_probability,  
                        embedding_output_dim=embedding_output_dim,
                        layer_sizes=layer_sizes,
                        activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


32 models


{'activation',
 'dropout_probability',
 'embedding_output_dim',
 'fraction_negative',
 'layer_sizes'}

In [12]:
reload(mhcflurry.class1_binding_predictor)
reload(mhcflurry)


<module 'mhcflurry' from '/Users/tim/sinai/git/mhcflurry/mhcflurry/__init__.pyc'>

In [16]:
# allele -> list of (train set, imputed train set, test set) for each fold
n_folds = 3
cv_splits = {}
for allele in alleles:
    print("Allele: %s" % allele)
    cv_iter = all_train_data.cross_validation_iterator(allele, n_folds=n_folds, shuffle=True)
    triples = []
    for (all_allele_train_split, test_split) in cv_iter:
        imputed_train_split = all_allele_train_split.impute_missing_values(
            fancyimpute.MICE(n_imputations=25, n_burn_in=5),
            min_observations_per_peptide=2,
            min_observations_per_allele=10)
        train_split = all_allele_train_split.groupby_allele_dictionary()[allele]
        triples.append((train_split, imputed_train_split, test_split))
    cv_splits[allele] = triples

Allele: HLA-A0201
Dropping 11505 peptides with <2 observations
Dropping 9 alleles with <10 observations: ['ELA-A1', 'HLA-B2701', 'HLA-B3508', 'HLA-B44', 'HLA-E0101', 'Mamu-B04', 'Patr-A0602', 'Patr-B0901', 'Patr-B1701']
[MICE] Completing matrix with shape (19142, 97)
[MICE] Starting imputation round 1/30, elapsed time 0.060
[MICE] Starting imputation round 2/30, elapsed time 5.034
[MICE] Starting imputation round 3/30, elapsed time 9.766
[MICE] Starting imputation round 4/30, elapsed time 15.046
[MICE] Starting imputation round 5/30, elapsed time 20.768
[MICE] Starting imputation round 6/30, elapsed time 26.491
[MICE] Starting imputation round 7/30, elapsed time 32.966
[MICE] Starting imputation round 8/30, elapsed time 39.923
[MICE] Starting imputation round 9/30, elapsed time 45.633
[MICE] Starting imputation round 10/30, elapsed time 50.197
[MICE] Starting imputation round 11/30, elapsed time 54.892
[MICE] Starting imputation round 12/30, elapsed time 59.063
[MICE] Starting imputati

In [15]:
cv_splits_without_imputation

{'HLA-A0201': [(Dataset(n=134465, alleles=['ELA-A1', 'Gogo-B0101', 'H-2-DB', 'H-2-DD', 'H-2-KB', 'H-2-KD', 'H-2-KK', 'H-2-LD', 'HLA-A0101', 'HLA-A0201', 'HLA-A0202', 'HLA-A0203', 'HLA-A0205', 'HLA-A0206', 'HLA-A0207', 'HLA-A0210', 'HLA-A0211', 'HLA-A0212', 'HLA-A0216', 'HLA-A0219', 'HLA-A0250', 'HLA-A0301', 'HLA-A0302', 'HLA-A11', 'HLA-A1101', 'HLA-A2', 'HLA-A2301', 'HLA-A2402', 'HLA-A2403', 'HLA-A2501', 'HLA-A26', 'HLA-A2601', 'HLA-A2602', 'HLA-A2603', 'HLA-A2902', 'HLA-A3001', 'HLA-A3002', 'HLA-A3101', 'HLA-A3201', 'HLA-A3301', 'HLA-A6601', 'HLA-A6801', 'HLA-A6802', 'HLA-A6901', 'HLA-A8001', 'HLA-B0702', 'HLA-B0801', 'HLA-B0802', 'HLA-B0803', 'HLA-B1402', 'HLA-B1501', 'HLA-B1502', 'HLA-B1503', 'HLA-B1509', 'HLA-B1517', 'HLA-B1801', 'HLA-B2701', 'HLA-B2702', 'HLA-B2703', 'HLA-B2705', 'HLA-B3501', 'HLA-B3503', 'HLA-B3508', 'HLA-B3801', 'HLA-B3901', 'HLA-B4001', 'HLA-B4002', 'HLA-B4201', 'HLA-B44', 'HLA-B4402', 'HLA-B4403', 'HLA-B4501', 'HLA-B4601', 'HLA-B4801', 'HLA-B5101', 'HLA-B5301'

In [ ]:
cv_df = defaultdict(list)
start = time.time()

for allele in train_data.alleles:
    print("Allele: %s" % allele)
    (subset_train, subset_impute, subset_test) = train_data.split_allele_randomly_and_impute_training_set(
        allele,
        (train_data._df.allele == allele).sum() // 3,
        imputation_method = fancyimpute.mice.MICE(),
    )
    
    X, Y, weights, original_peptides = get_shuffled_fields(data)
            
    cv = sklearn.cross_validation.LabelKFold(original_peptides, n_folds = 3)
    for (fold_num, (train_indices, test_indices)) in enumerate(cv):
        print("-- fold #%d/3" % (fold_num + 1,))
        X_cv_train = X[train_indices]
        X_cv_test = X[test_indices]
        
        Y_cv_train = Y[train_indices]
        Y_cv_test = Y[test_indices]
        
        weights_cv_train = weights[train_indices]
        weights_cv_test = weights[test_indices]
        
        original_peptides_train = original_peptides[train_indices]
        original_peptides_test = original_peptides[test_indices]
        impute = False
        
        np.random.shuffle(models_params_list)
        for (i, original_model_params) in enumerate(models_params_list):
            model_params = dict(original_model_params)
            fraction_negative = model_params["fraction_negative"]
            del model_params["fraction_negative"]
            
            print("%10s fold %3d [%3d / %3d] train_size=%d test_size=%d impute=%s model=%s" %
                  (allele, fold_num, i, len(models_params_list), len(train_indices), len(test_indices), impute, original_model_params))
            sys.stdout.flush()
            
            
            predictor = mhcflurry.Class1BindingPredictor.from_hyperparameters(
                max_ic50=max_ic50,
                **model_params)

            fit_time = -time.time()
            predictor.fit(
                X_cv_train,
                Y_cv_train,
                sample_weights=weights_cv_train,
                verbose=False,
                batch_size=128,
                n_training_epochs=250,
                n_random_negative_samples=int(fraction_negative * len(Y_cv_train))
            )
            fit_time += time.time()
    
            Y_cv_train_9mer_predictions = predictor.predict(X_cv_train)
            Y_cv_test_9mer_predictions = predictor.predict(X_cv_test)
            
            Y_train_true, Y_train_pred = collapse_9mer_affinities(
                Y_9mer_true=Y_cv_train,
                Y_9mer_pred=Y_cv_train_9mer_predictions,
                original_peptides=original_peptides_train)
            
            Y_test_true, Y_test_pred = collapse_9mer_affinities(
                Y_9mer_true=Y_cv_test,
                Y_9mer_pred=Y_cv_test_9mer_predictions,
                original_peptides=original_peptides_test)
            
            cv_df["allele"].append(allele)
            cv_df["allele_size"].append(len(Y))
            cv_df["train_size"].append(len(Y_cv_train))
            cv_df["model_params"].append(model_params)
            cv_df["fit_time"].append(fit_time)

            for (param, param_value) in model_params.items():
                cv_df[param].append(param_value)
            
            for (key, value) in make_scores(Y_train_true, Y_train_pred).items():
                cv_df["train_%s" % key].append(value)
                print("train %s: %f" % (key, value))
            
            for (key, value) in make_scores(
                    Y_test_true, 
                    Y_test_pred).items():
                cv_df["test_%s" % key].append(value)
                print("test %s: %f" % (key, value))


cv_df = pandas.DataFrame(cv_df)
cv_df["layer0_size"] = [x[0] for x in cv_df.layer_sizes]
print(time.time() - start)
cv_df

In [ ]:
cv_df["combined"] = cv_df.test_auc + cv_df.test_f1 + cv_df.test_tau

In [ ]:
cv_df_str = cv_df.copy()
print(cv_df_str.columns)
del cv_df_str['model_params']
del cv_df_str['fit_time']

for col in ["layer_sizes"]:
    cv_df_str[col] = [str(x) for x in cv_df_str[col]]
summary = cv_df_str.groupby(list(cv_df_str.columns[:6])).mean() #.reset_index()
summary.sort("combined", ascending=False, inplace=True)
summary.to_csv("../data/cv_hla0201_summary_128.csv")

In [ ]:
cv_df.sort("combined", ascending=False, inplace=True)
cv_df

In [ ]:
cv_df = pandas.DataFrame(cv_df)
cv_df["layer0_size"] = [x[0] for x in cv_df.layer_sizes]
cv_df

In [ ]:
cv_df.to_csv("cv5.csv")

In [ ]:
group_columns = ["allele", "allele_size", "impute"]
group_columns.extend(models_params_explored)
group_columns.append("layer0_size")
group_columns.remove("layer_sizes")
print(mean_with_std(cv_df.groupby(group_columns).test_auc)) #.sort(inplace=False, ascending=False)



In [ ]:
def best_by(score):
    means = cv_df.groupby(group_columns)[score].mean().reset_index()
    max_rows = []
    for allele in means.allele.unique():
        max_rows.append(means.ix[means.allele == allele][score].argmax())
    return means.ix[max_rows]

In [ ]:
best_by('test_auc')


In [ ]:
best_by('test_tau')

In [ ]:
best_by('test_f1')